<h2><a href="https://leetcode.com/problems/find-first-and-last-position-of-element-in-sorted-array">34. Find First and Last Position of Element in Sorted Array</a></h2><h3>Medium</h3><hr><p>Given an array of integers <code>nums</code> sorted in non-decreasing order, find the starting and ending position of a given <code>target</code> value.</p>

<p>If <code>target</code> is not found in the array, return <code>[-1, -1]</code>.</p>

<p>You must&nbsp;write an algorithm with&nbsp;<code>O(log n)</code> runtime complexity.</p>

<p>&nbsp;</p>
<p><strong class="example">Example 1:</strong></p>
<pre><strong>Input:</strong> nums = [5,7,7,8,8,10], target = 8
<strong>Output:</strong> [3,4]
</pre><p><strong class="example">Example 2:</strong></p>
<pre><strong>Input:</strong> nums = [5,7,7,8,8,10], target = 6
<strong>Output:</strong> [-1,-1]
</pre><p><strong class="example">Example 3:</strong></p>
<pre><strong>Input:</strong> nums = [], target = 0
<strong>Output:</strong> [-1,-1]
</pre>
<p>&nbsp;</p>
<p><strong>Constraints:</strong></p>

<ul>
	<li><code>0 &lt;= nums.length &lt;= 10<sup>5</sup></code></li>
	<li><code>-10<sup>9</sup>&nbsp;&lt;= nums[i]&nbsp;&lt;= 10<sup>9</sup></code></li>
	<li><code>nums</code> is a non-decreasing array.</li>
	<li><code>-10<sup>9</sup>&nbsp;&lt;= target&nbsp;&lt;= 10<sup>9</sup></code></li>
</ul>


## Solution 1 — Single binary search with left/right bias (one function with a flag)

Intuition and approach:
- We use binary search to locate the target. To find both first and last positions with binary search we can run a binary search twice: once biased to the left (to find the first occurrence) and once biased to the right (to find the last occurrence).
- The `leftBias` flag controls which direction we continue searching after finding an occurrence: move left to find earlier occurrences (left bias) or move right to find later occurrences (right bias).

Detailed code explanation (line-by-line):
- `low = 0`, `high = len(nums)-1`: inclusive search range.
- `result = -1`: stores index when target is found; stays -1 if not found.
- Loop `while low <= high:`: standard inclusive binary search loop.
- `mid = (low + high) // 2`: integer midpoint.
- If `target > nums[mid]`: target is to the right, set `low = mid + 1`.
- Elif `target < nums[mid]`: target is to the left, set `high = mid - 1`.
- Else (nums[mid] == target): set `result = mid` (record an occurrence). Now decide direction: if `leftBias` then `high = mid - 1` to continue searching left for the first occurrence; else `low = mid + 1` to continue searching right for the last occurrence.
- After the loop return `result` which is the found left/right index or -1 if never found.

### Dry run (step-by-step) — example: nums = [5,7,7,8,8,10], target = 8
We run the same binSearch twice: first with leftBias=True (to get left index), then with leftBias=False (to get right index).
-- Left-biased search (find first occurrence):
1) low=0, high=5 -> mid=(0+5)//2=2, nums[2]=7. target(8) > 7 -> low=mid+1=3. result=-1.
2) low=3, high=5 -> mid=(3+5)//2=4, nums[4]=8. nums[mid]==target -> result=4. leftBias True so high=mid-1=3 (continue left).
3) low=3, high=3 -> mid=3, nums[3]=8. match -> result=3. leftBias True so high=mid-1=2.
Loop ends (low=3, high=2). Return result=3 — first occurrence index.

-- Right-biased search (find last occurrence):
1) low=0, high=5 -> mid=2, nums[2]=7. target>7 -> low=3. result=-1.
2) low=3, high=5 -> mid=4, nums[4]=8. match -> result=4. leftBias False so low=mid+1=5 (continue right).
3) low=5, high=5 -> mid=5, nums[5]=10. target<10 -> high=mid-1=4.
Loop ends (low=5, high=4). Return result=4 — last occurrence index.

Edge cases and how the code handles them:
- Empty array: high = -1 initially; loop skipped; result stays -1 (correct).
- Single-element array: works normally; if equal returns 0 else -1.
- Target not present: result remains -1. Both left/right calls return -1. The main `searchRange` returns [-1,-1].
- All elements equal to target: left-biased search returns 0, right-biased returns len(nums)-1.
- Negative numbers and zeros: unaffected; binary search only relies on ordering.

Time complexity (exact):
- Each binary search call runs in O(log n). We make two calls (left and right) -> overall O(log n) + O(log n) = O(log n).
- Iteration bound per call: at most ⌈log2(n+1)⌉ iterations (each iteration O(1) work).
Space complexity: O(1) extra space (a few integer variables).

How this differs from a 'normal' binary search and why it works:
- Normal binary search stops when it finds any matching index. To find the first/last occurrence specifically, we must continue searching after finding a match — by moving the `high` pointer left to search earlier elements (for first), or moving the `low` pointer right to search later elements (for last).
- The key change is: record the found index (`result = mid`) but continue binary search in a chosen direction instead of returning immediately. This keeps the O(log n) complexity while locating boundary occurrences.

In [39]:
from typing import List
class Solution:
    def searchRange(self, nums: List[int], target: int) -> List[int]:
        left = self.binSearch(nums, target, True)
        right = self.binSearch(nums, target, False)
        return [left, right]

    # leftBias=[True/False], if false, res is rightBiased
    def binSearch(self, nums, target, leftBias):
        low = 0
        high = len(nums) - 1
        result = -1
        while low <= high:
            mid = (low + high) // 2
            if target > nums[mid]:
                low = mid + 1
            elif target < nums[mid]:
                high = mid - 1
            else:
                result = mid
                if leftBias:
                    high = mid - 1
                else:
                    low = mid + 1
        return result

nums = [2,2] 
target = 2
s = Solution()
ans = s.searchRange(nums,target)
print(f"For {nums} array the target {target} starts and end at:{ans}")

For [2, 2] array the target 2 starts and end at:[0, 1]


## Solution 2 — Separate left and right helper binary searches (explicit functions)

Intuition and approach:
- This variant splits responsibilities: one helper finds the left-most occurrence, the other finds the right-most occurrence. Each helper is a standard binary search modified to continue searching in a direction when a match is found.
- Splitting improves clarity and makes the intention explicit: `find_left_occurence` returns -1 if not found, otherwise the first index; `find_right_occurence` returns the last index.

Detailed code explanation (line-by-line):
- `find_left_occurence`: initialize `start=0, end=len(nums)-1, left_occurence=-1`. Loop while `start<=end`. Compute `mid`. If `nums[mid]==target`: set `left_occurence=mid` and `end=mid-1` to search left. If `nums[mid]<target`: `start=mid+1`. Else `end=mid-1`. After loop return `left_occurence`.
- `find_right_occurence`: symmetric: when `nums[mid]==target` record `right_occurence=mid` and `start=mid+1` to search right for later occurrences.
- `searchRange` first calls `find_left_occurence`. If that returns -1, the overall function returns [-1,-1] immediately (optimization). Otherwise it calls `find_right_occurence` and returns both results.

Extreme dry run (very detailed) — example: nums = [5,7,7,8,8,10], target = 8

-- find_left_occurence:
1) start=0,end=5 -> mid=2 nums[2]=7 < 8 -> start=3, left_occurence=-1.
2) start=3,end=5 -> mid=4 nums[4]=8 == target -> left_occurence=4, end=mid-1=3.
3) start=3,end=3 -> mid=3 nums[3]=8 == target -> left_occurence=3, end=2.
Loop ends -> return left_occurence=3.

-- find_right_occurence:
1) start=0,end=5 -> mid=2 nums[2]=7 < 8 -> start=3, right_occurence=-1.
2) start=3,end=5 -> mid=4 nums[4]=8 == target -> right_occurence=4, start=mid+1=5.
3) start=5,end=5 -> mid=5 nums[5]=10 > 8 -> end=mid-1=4.
Loop ends -> return right_occurence=4.

Edge cases and behavior details:
- Empty array: left returns -1 => overall [-1,-1].
- Single-element matching array: left and right both return 0.
- Target absent: left returns -1 and short-circuits overall result to [-1,-1].
- Duplicate range: left finds earliest index; right finds latest index. Both are safe for very large arrays because they use constant extra space.

Complexity analysis:
- Each helper is O(log n). We may call both helpers in the worst case -> total O(log n) + O(log n) = O(log n).
- Space: O(1) extra space.

Difference from normal binary search:
- Same idea as solution 1: after finding a match, continue moving the appropriate bound instead of returning immediately. The split into separate functions simply makes each responsibility explicit.

In [ ]:
from typing import List
class Solution:
    def searchRange(self, nums: List[int], target: int) -> List[int]:
        left_occurence = self.find_left_occurence(nums, target)

        if left_occurence == -1:
            return [-1, -1]

        right_occurence = self.find_right_occurence(nums, target)
        return [left_occurence, right_occurence]
    
    def find_left_occurence(self, nums: List[int], target: int) -> int:
        start = 0
        end = len(nums) - 1
        left_occurence = -1
        
        while start <= end:
            mid = start + (end - start) // 2
            if nums[mid] == target:
                left_occurence, end = mid, mid - 1
            elif nums[mid] < target:
                start = mid + 1
            else:
                end = mid - 1
        
        return left_occurence

    def find_right_occurence(self, nums: List[int], target: int) -> int:
        start = 0
        end = len(nums) - 1
        right_occurence = -1
        
        while start <= end:
            mid = start + (end - start) // 2
            if nums[mid] == target:
                right_occurence, start = mid, mid + 1
            elif nums[mid] < target:
                start = mid + 1
            else:
                end = mid - 1
        return right_occurence

nums = [2,2] 
target = 2
s = Solution()
ans = s.searchRange(nums,target)
print(f"For {nums} array the target {target} starts and end at:{ans}")

## Solution 3 — Linear scan (simple iteration)

Intuition and approach:
- Iterate through the array once, tracking the first index where `nums[i] == target` and the last index seen. This is straightforward and easy to implement but not logarithmic.

Code explanation (detailed):
- `l = len(nums)`; initialize `first = -1`, `last = -1`.
- Iterate `for i in range(l)`: if `nums[i] == target`: if `first == -1` set `first = i`; always set `last = i` when a match is found.
- After loop return `[first, last]` (will be [-1, -1] if never found).

Extreme dry run (detailed) — nums = [5,7,7,8,8,10], target = 8
1) i=0 nums[0]=5 != 8 -> first=-1,last=-1.
2) i=1 nums[1]=7 != 8 -> first=-1,last=-1.
3) i=2 nums[2]=7 != 8 -> first=-1,last=-1.
4) i=3 nums[3]=8 == target -> first = 3 (since first was -1), last = 3.
5) i=4 nums[4]=8 == target -> first = 3, last = 4.
6) i=5 nums[5]=10 != 8 -> first = 3, last = 4.
Return [3,4].

Edge cases / behavior:
- Empty array: loop skipped, return [-1,-1].
- Single element arrays: works correctly but still O(1) time for length 1.
- Works with duplicates (tracks first and last).

Time and space complexity calculation (exact):
- Time: O(n) because we iterate over every element once in the worst case. For length n that's n iterations and O(1) work per iteration -> exact operations ~ n comparisons plus a few assignments.
- Space: O(1) extra space for `first`, `last`, and loop index.

When to use this approach:
- Use when n is small or when simplicity is preferred and O(n) is acceptable. If strict O(log n) is required (as in the problem statement), prefer the binary-search solutions.

In [33]:
from typing import List
class Solution:
    def searchRange(self, nums: List[int], target: int) -> List[int]:
        l = len(nums)
        first = -1
        last = -1
        for i in range(l):
            if nums[i] == target:
                if first == -1:
                    first = i
                last = i
        return [first,last]

nums = [5,7,7,8,8,10] 
target = 8
s = Solution()
ans = s.searchRange(nums,target)
print(f"For {nums} array the target {target} starts and end at:{ans}")

nums1 = [5,7,7,8,8,10]
target1 = 6
s1 = Solution()
ans1 = s1.searchRange(nums1,target1)
print(f"For {nums1} array the target {target1} starts and end at:{ans1}")


For [5, 7, 7, 8, 8, 10] array the target 8 starts and end at:[3, 4]
For [5, 7, 7, 8, 8, 10] array the target 6 starts and end at:[-1, -1]


## Comparison, pros/cons and final notes

Summary of approaches:
- Solution 1: Single binary search function with a `leftBias` flag. Runs two binary searches (left and right bias). Time O(log n). Space O(1). Good balance between code reuse and clarity.
- Solution 2: Two explicit helper functions (`find_left_occurence` and `find_right_occurence`). Equally efficient: two O(log n) searches -> overall O(log n). Slightly clearer intent and easier to test individually.
- Solution 3: Linear scan. Time O(n), Space O(1). Simple to implement but does not meet the problem's O(log n) requirement for large n.

Exact complexity values (for array length n):
- Binary-search solutions: at most 2 * ⌈log2(n+1)⌉ iterations total across the two searches. Each iteration is O(1) so exact upper bound on primitive operations is c * 2 * ⌈log2(n+1)⌉ for some small constant c. This is asymptotically O(log n).
- Linear scan: exactly n iterations in worst case (one comparison per element), O(n) time.
- Space for all solutions: O(1) extra space.

Why modified binary search differs from normal binary search:
- Normal binary search stops when it finds any occurrence and returns. Modified boundary searches must continue after finding a match to search for earlier or later occurrences. We do this by recording the current `mid` into a result variable and moving the search boundary (`high` or `low`) to continue searching inside the remaining half. This minor change preserves the logarithmic iteration count but yields boundary indices instead of an arbitrary matching index.

Recommendation:
- For this LeetCode problem prefer Solution 2 (clear intent) or Solution 1 (compact reuse). Use Solution 3 only if O(n) is acceptable or for quick prototyping.

Testing suggestions:
- Test empty input `[]`, single-element arrays, arrays where all elements equal target, arrays where target is not present, target smaller than all elements, and target larger than all elements.
- Consider writing short unit tests using `pytest` to cover these cases and validate both binary-search approaches return identical outputs.